In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
!pip3 install beautifulsoup4 nltk pymorphy2

You are using pip version 8.1.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [51]:
from bs4 import BeautifulSoup

html = """
<!DOCTYPE html>
<html>
<body>

<p class="class1">This is a paragraph.</p>
<p id="id1">This is a paragraph.</p>
<p>This is a paragraph.</p>

</body>
</html>
"""

bs = BeautifulSoup(html)
all_p_tags = bs.find_all("p")
print("tags found:", all_p_tags)
extracted_text = " ".join(map(lambda x: x.text, all_p_tags))
print("text extracted: '{}'".format(extracted_text))

tags found: [<p class="class1">This is a paragraph.</p>, <p id="id1">This is a paragraph.</p>, <p>This is a paragraph.</p>]
text extracted: 'This is a paragraph. This is a paragraph. This is a paragraph.'


C:\Users\Evgenii\AppData\Local\Programs\Python\Python36-32\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Evgenii\AppData\Local\Programs\Python\Python36-32\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html.parser")

  markup_type=markup_type))


In [4]:
import re
cleaned_text = re.sub("[^a-zA-Z0-9 ]", "", extracted_text)
print("extracted text: '{}'".format(extracted_text))
print("cleaned text: '{}'".format(cleaned_text))

extracted text: 'This is a paragraph. This is a paragraph. This is a paragraph.'
cleaned text: 'This is a paragraph This is a paragraph This is a paragraph'


In [10]:
from nltk.stem import PorterStemmer

words = ['caresses', 'flies', 'dies', 'mules', 'denied', 'died', 'agreed', 'owned', 
         'humbled', 'sized', 'meeting', 'stating', 'siezing', 'itemization', 'sensational', 
         'traditional', 'reference', 'colonizer', 'plotted']

ps = PorterStemmer()
for word in words: 
    print(word, "->", ps.stem(word))

caresses -> caress
flies -> fli
dies -> die
mules -> mule
denied -> deni
died -> die
agreed -> agre
owned -> own
humbled -> humbl
sized -> size
meeting -> meet
stating -> state
siezing -> siez
itemization -> item
sensational -> sensat
traditional -> tradit
reference -> refer
colonizer -> colon
plotted -> plot


In [58]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()
forms = morph.parse(u"лук")
print(forms)

[Parse(word='лук', tag=OpencorporaTag('NOUN,inan,femn plur,gent'), normal_form='лука', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'лук', 43, 8),)), Parse(word='лук', tag=OpencorporaTag('NOUN,inan,masc sing,nomn'), normal_form='лук', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'лук', 330, 0),)), Parse(word='лук', tag=OpencorporaTag('NOUN,inan,masc sing,accs'), normal_form='лук', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'лук', 330, 4),))]


In [59]:
print(forms[0].normal_form)
print(forms[0].score)

лука
0.3333333333333333


### Задание

С какой вероятностью слово "стали" в тексте является формой слова "сталь"?

In [19]:
df = pd.read_csv("dataset1.tsv", sep="\t")
df.head()

,positive,text
0,1,this was the first clive cussler i've ever rea...
1,1,i liked the Da Vinci Code a lot.
2,1,i liked the Da Vinci Code a lot.
3,1,I liked the Da Vinci Code but it ultimatly did...
4,1,that's not even an exaggeration ) and at midni...


In [20]:
from sklearn.feature_extraction.text import CountVectorizer

"""
Convert a collection of text documents to a matrix of token counts

This implementation produces a sparse representation of the counts using
scipy.sparse.coo_matrix.

Parameters
----------
input : string {'filename', 'file', 'content'}

encoding : string, 'utf-8' by default.

analyzer : string, {'word', 'char', 'char_wb'} or callable

ngram_range : tuple (min_n, max_n)

stop_words : string {'english'}, list, or None (default)

max_features : int or None, default=None
"""

cv = CountVectorizer(input="content", analyzer="word")
BoW_matrix = cv.fit_transform(df.text)
BoW_matrix

<6917x2132 sparse matrix of type '<class 'numpy.int64'>'
	with 65390 stored elements in Compressed Sparse Row format>

In [21]:
from sklearn.feature_extraction.text import TfidfTransformer

"""
Parameters
----------
norm : 'l1', 'l2' or None, optional

use_idf : boolean, default=True
"""

tfidfer = TfidfTransformer()
tfidfed_matrix = tfidfer.fit_transform(BoW_matrix)
tfidfed_matrix

<6917x2132 sparse matrix of type '<class 'numpy.float64'>'
	with 65390 stored elements in Compressed Sparse Row format>

In [23]:
from sklearn.feature_extraction.text import FeatureHasher

"""
Parameters
----------
n_features : integer, optional
    The number of features (columns) in the output matrices. Small numbers
    of features are likely to cause hash collisions, but large numbers
    will cause larger coefficient dimensions in linear learners.
input_type : string, optional, default "dict"
    Either "dict" (the default) to accept dictionaries over
    (feature_name, value); "pair" to accept pairs of (feature_name, value);
    or "string" to accept single strings.
    feature_name should be a string, while value should be a number.
    In the case of "string", a value of 1 is implied.
    The feature_name is hashed to find the appropriate column for the
    feature. The value's sign might be flipped in the output (but see
    non_negative, below).
"""

fh = FeatureHasher(input_type="string")
hashed = fh.fit_transform(map(lambda x: x.split(), df.text))
hashed

<6917x1048576 sparse matrix of type '<class 'numpy.float64'>'
	with 71991 stored elements in Compressed Sparse Row format>

In [42]:
df = pd.read_csv("dataset1.tsv", sep="\t")
df.head()

,positive,text
0,1,this was the first clive cussler i've ever rea...
1,1,i liked the Da Vinci Code a lot.
2,1,i liked the Da Vinci Code a lot.
3,1,I liked the Da Vinci Code but it ultimatly did...
4,1,that's not even an exaggeration ) and at midni...


In [43]:
labels = df["positive"]
texts = df["text"]

In [45]:
%%time

from nltk.stem import PorterStemmer

ps = PorterStemmer()

def clean_text(text):
    regex = "[^a-zA-Z0-9 ]"
    cleaned_text = re.sub(regex, "", text).lower()
    
    words = cleaned_text.split()
    stemmed_words = []
    
    for word in words:
        stemmed_words.append(ps.stem(word))
    
    return " ".join(stemmed_words)

texts = texts.apply(clean_text)

Wall time: 1.5 s


In [46]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = ...
bow_matrix = vectorizer.fit_transform(texts)

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

lr = ...

print(cross_val_score(lr, bow_matrix, labels, scoring="accuracy").mean())

0.9535870773780744


## Резюме

- Извлечение текстов с веб-страниц - BeautifulSoup
- Удаление пунктуации и прочих символов  - с помощью регулярных выражений
- Для английских текстов - стемминг
- Для русских текстов - морфология и приведение к начальной форме
- Создание признаков с помощью Bag of Words & TF-IDF
- Лучше всего использовать линейные модели - они справляются с большим количеством признаков